In [3]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import time
from sumo_rl import SumoEnvironment

In [4]:
print("SUMO HOME:", os.environ.get("SUMO_HOME"))

SUMO HOME: /opt/miniconda3/lib/python3.13/site-packages/sumo


In [5]:
# File paths
NET_FILE = 'single-intersection.net.xml'      # network file
ROU_FILE = 'single-intersection-vertical.rou.xml'  # route file (traffic flows)

# Initialize SUMO environment
env = SumoEnvironment(
    net_file=NET_FILE,
    route_file=ROU_FILE,
    use_gui=True,                 # show SUMO GUI
    single_agent=True,            # one intersection controller
    reward_fn='diff-waiting-time',
    out_csv_name='outputs/traffic_log'
)


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      


In [6]:
# ===== Check the actions and the state space =====
print("Action Space:", env.action_space)
print("State Space:", env.observation_space)

Action Space: Discrete(4)
State Space: Box(0.0, 1.0, (21,), float32)


In [ ]:
# ========= Simple simulation test of  SUMO-RL ===========

# Intialization
obs, info = env.reset()
avg_speed_per_road, avg_wait_time_per_road = np.zeros(4), np.zeros(4)
avg_pressure, avg_total_speed = [],[]
rewards, actions = [], []
sim_time = 0.0

for step in range(100):

    print(f"--- Step {step} / Time {sim_time:.1f} ---")

    # Take a random action
    action = env.action_space.sample()
    actions.append(action)

    # Recieve the env feedback
    obs, reward, done, truncated, info = env.step(action)

    # Current simulation time
    sim_time = env.sumo.simulation.getTime()

    # Get the vehicles IDs
    veh_ids = env.sumo.vehicle.getIDList()

    # Collisions Detection
    collisions = env.sumo.simulation.getCollidingVehiclesIDList()
    if len(collisions) > 0:
        print(f"Collisions: {collisions}")

        # Reset env after collision
        obs, info = env.reset()
        print("Environment reset.\n")
        continue

    # Integrate information from the environment
    lane_counter = {}
    total_speed = 0.0
    intsec_pressure = 0.0

    for vid in veh_ids:
        speed = env.sumo.vehicle.getSpeed(vid)
        lane = env.sumo.vehicle.getLaneID(vid)
        # pos = env.sumo.vehicle.getPosition(vid)
        # print(f"{vid}: {speed:.2f} m/s on {edge} ({lane}), pos={pos}")
        total_speed += speed
        lane_counter[lane] =  lane_counter.get(lane,0) + 1
    
    avg_speed = total_speed/len(veh_ids) if len(veh_ids) > 0 else 0
    avg_total_speed.append(avg_speed)

    print(f"Average speed of all vehicles: {avg_speed:.2f} m/s")
    print(f"Vechicles per lane:{lane_counter}")


    # Arrived vehicles
    # arrived = env.sumo.simulation.getArrivedIDList()
    # if len(arrived) > 0:
    #     print(f"Arrived: {arrived}")

    # Red-Green Light States
    # for tls in env.ts_ids:
    state = env.sumo.trafficlight.getRedYellowGreenState(env.ts_ids[0])
    phase = env.sumo.trafficlight.getPhase(env.ts_ids[0])
    print(f"TraLight state=:{state} ")

    # Reward
    print(f"Step reward: {reward:.3f}\n")

    time.sleep(0.2)

    if done:
        obs, info = env.reset()
        print("Environment reset.\n")

env.close()
print("Simulation finished!")


 Retrying in 1 seconds
--- Step 0 / Time 0.0 ---
Average speed of all vehicles: 12.61 m/s
Vechicles per lane:{'e_t_0': 2, 'e_t_1': 1, 'n_t_1': 1, 'n_t_0': 2, 's_t_0': 2, 's_t_1': 1, 'w_t_0': 2, 'w_t_1': 1}
TraLight state=:GGrrrrGGrrrr 
Step reward: 0.000

--- Step 1 / Time 5.0 ---
Average speed of all vehicles: 12.36 m/s
Vechicles per lane:{'e_t_0': 2, 'e_t_1': 1, 'n_t_1': 1, 'n_t_0': 2, 's_t_0': 2, 's_t_1': 1, 'w_t_0': 2, 'w_t_1': 1}
TraLight state=:GGrrrrGGrrrr 
Step reward: 0.000

--- Step 2 / Time 10.0 ---
Average speed of all vehicles: 6.18 m/s
Vechicles per lane:{'e_t_0': 2, ':t_5_0': 1, 'n_t_1': 1, 't_s_0': 1, 'n_t_0': 3, 's_t_0': 3, 't_n_0': 1, 's_t_1': 1, 'w_t_0': 2, 't_n_1': 1}
TraLight state=:rrrrrGrrrrrG 
Step reward: -0.010

--- Step 3 / Time 15.0 ---
Average speed of all vehicles: 7.31 m/s
Vechicles per lane:{'e_t_0': 2, 't_s_1': 1, 'n_t_1': 2, 't_s_0': 1, 'n_t_0': 3, 's_t_0': 3, 't_n_0': 1, 's_t_1': 2, 'w_t_0': 2, 't_n_1': 1}
TraLight state=:rrrrrGrrrrrG 
Step reward: -0